In [1]:
import re
from functools import reduce

import pandas as pd

# 경기도_시군구별_인구수_2019

In [2]:
gg_pop = pd.read_csv('./data/clustering_data/경기도 시군별 성별 인구.csv', encoding='cp949')

In [4]:
gg_pop.rename(columns = {'행정구역(시군구)별' : '시군', '총인구수 (명)' :'총인구수'}, inplace = True)

# 경기지역화폐 시군별 특징

In [6]:
gg_lc_feature = pd.read_csv('./data/clustering_data/경기지역화폐 시군별 특징.csv')

In [7]:
gg_lc_feature['개시일'] = gg_lc_feature['시군(개시일)'].apply(lambda x : x.split('(')[1][:-1])
gg_lc_feature['시군(개시일)'] = gg_lc_feature['시군(개시일)'].apply(lambda x : x.split('(')[0])
gg_lc_feature['할인율_상시'] = gg_lc_feature['할인율'].apply(lambda x : int(re.findall('\d+', x)[0]))
gg_lc_feature['할인율_한시'] = gg_lc_feature['할인율'].apply(lambda x : int(re.findall('\d+', x)[1]))
gg_lc_feature.drop('할인율', axis=1, inplace=True)

In [8]:
gg_lc_feature['할인율_상시'] = gg_lc_feature['할인율_상시'].apply(lambda x: float(x)*0.01 if 1<x<11 else None)
gg_lc_feature['할인율_한시'] = gg_lc_feature['할인율_한시'].apply(lambda x: float(x)*0.01 if 1<x<11 else None)
gg_lc_feature['구매한도_월'] = gg_lc_feature['구매한도(월/연,만원)'].apply(lambda x : x.split('/')[0])
gg_lc_feature['구매한도_연'] = gg_lc_feature['구매한도(월/연,만원)'].apply(lambda x : x.split('/')[1])
gg_lc_feature.drop(['구매한도(월/연,만원)','비고','개시일'], axis=1, inplace=True)
gg_lc_feature.rename(columns = {'시군(개시일)' : '시군'}, inplace = True)

# 경기지역화폐 발행 및 이용현황

In [9]:
gg_lc_current_state = pd.read_csv('./data/clustering_data/경기지역화폐 발행 및 이용현황.csv', encoding='cp949')

In [10]:
gg_lc_current_state['년도'] = gg_lc_current_state['기준년월'].apply(lambda x : x.split('-')[0])
gg_lc_current_state_2019 = gg_lc_current_state[gg_lc_current_state['년도'] == '2019']

In [11]:
sum_gg_lc = gg_lc_current_state_2019.groupby('시군명').sum().fillna(0)
sum_gg_lc.reset_index(level=['시군명'], inplace = True)
sum_gg_lc.rename(columns = {'시군명' : '시군'}, inplace = True)

In [12]:
sum_gg_lc['지역화폐사용액(천원)'] = sum_gg_lc['월별카드사용액(천원)'] + sum_gg_lc['월별모바일사용액(천원)'] + sum_gg_lc['월별지류상품권회수액(천원)']

In [13]:
sum_gg_lc_extract = sum_gg_lc[['시군', '지역화폐사용액(천원)']]

# 경기도 시군별 경제활동인구

In [14]:
gg_economic_pop_2019 = pd.read_csv('./data/clustering_data/경기도 시군별 경제활동인구.csv', encoding='cp949')

In [15]:
gg_economic_pop_rate = gg_economic_pop_2019[['행정구역별','경제활동참가율 (％)']]
gg_economic_pop_rate.rename(columns = {'행정구역별' : '시군','경제활동참가율 (％)' : '경제활동참가율'}, inplace = True)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# merge

In [16]:
dfs = [gg_pop, sum_gg_lc_extract, gg_economic_pop_rate]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='시군', how='left'), dfs)[1:]

# 주택비율과 연령대비율

In [17]:
gg_house_age = pd.read_csv('./data/clustering_data/경기_주택과연령비율.csv')

In [18]:
gg_house_age.drop('Unnamed: 0',axis=1, inplace = True)

# 최종 클러스터링 데이터

In [21]:
gg_e_index = pd.read_csv('./data/clustering_data/경기_e지표_정리.csv')

In [22]:
data = pd.merge(df_merge, gg_e_index, left_on="시군", right_on="지역별").fillna(0)
data.drop(['Unnamed: 0','지역별','인구총조사 인구'], axis=1, inplace = True)

In [23]:
data = pd.merge(data, gg_house_age, left_on="시군", right_on="행정구역별(시군구)", how='outer') # 주택과 연령대 추가
data.drop('행정구역별(시군구)',axis=1, inplace = True)

In [24]:
data

,시군,총인구수,지역화폐사용액(천원),경제활동참가율,서비스업사업체비율,제조업사업체비율,도소매업사업체비율,운수업사업체비율,제조업종사자비율,도소매업종사자비율,...,합계출산율,도시지역면적,주택,전체대비주택비율,면적대비주택비율,미성년비율,청년비율,중년비율,장년비율,노년비율
0,수원시,1194465,22696236.0,62.6,0.759848,0.052995,0.234011,0.105050,0.106625,0.130200,...,0.890,121182030,354566,0.086990,0.002926,0.191714,0.217396,0.260891,0.226560,0.103439
1,성남시,942724,86074070.0,62.0,0.767902,0.063562,0.249546,0.090824,0.085411,0.159631,...,0.852,141820898,251281,0.061650,0.001772,0.172351,0.198688,0.253128,0.241703,0.134130
2,의정부시,451868,9062650.0,59.4,0.758880,0.049615,0.233721,0.114696,0.058894,0.163065,...,0.847,81598380,142635,0.034994,0.001748,0.177133,0.187298,0.244418,0.248429,0.142722
3,안양시,567044,20391808.0,62.1,0.733692,0.086717,0.263631,0.091067,0.135426,0.155778,...,0.980,58480054,171744,0.042136,0.002937,0.173936,0.200810,0.241278,0.258014,0.125962
4,부천시,829996,31543254.0,63.0,0.660901,0.173820,0.221111,0.093887,0.233154,0.142733,...,0.807,53450000,255150,0.062599,0.004774,0.165990,0.204109,0.242981,0.263626,0.123295
5,광명시,316552,6372380.0,61.8,0.710427,0.092470,0.258978,0.122433,0.162044,0.187550,...,0.840,38506473,99721,0.024466,0.002590,0.186101,0.182426,0.259265,0.242419,0.129789
6,평택시,513027,10543560.0,63.8,0.722968,0.105638,0.229402,0.094678,0.382154,0.111615,...,1.102,151090527,171851,0.042162,0.001137,0.198596,0.193366,0.265432,0.224345,0.118261
7,동두천시,94768,2106819.0,56.2,0.724726,0.077040,0.217266,0.107948,0.202095,0.126663,...,0.854,33565811,33510,0.008221,0.000998,0.171149,0.161159,0.226589,0.252754,0.188349
8,안산시,650918,27036345.0,63.4,0.635912,0.173842,0.213921,0.116399,0.391430,0.114223,...,0.887,151472339,193891,0.047569,0.001280,0.176492,0.215499,0.246811,0.261677,0.099521
9,고양시,1066351,28739326.0,59.7,0.734979,0.084586,0.266667,0.104599,0.098133,0.197817,...,0.802,194127720,313254,0.076854,0.001614,0.182635,0.184406,0.253629,0.251385,0.127944


In [25]:
data.to_csv('./data/cluster_data.csv', sep=',', index=False)